In [1]:
!pip install kagglehub
import kagglehub
path = kagglehub.dataset_download("phhasian0710/seed-iv")
print("Path to dataset files:", path)

100%|██████████| 13.7G/13.7G [02:34<00:00, 95.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1


In [27]:
import os
for root, dirs, files in os.walk(path):
    for dir in dirs:
        print(os.path.join(root, dir))  # Print directories
    for file in files:
        print(os.path.join(root, file))  # Print files

/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eye_raw_data
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eye_feature_smooth
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_raw_data
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/ReadMe.txt
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/Channel Order.xlsx
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv/eye_raw_data
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv/eye_feature_smooth
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv/eeg_raw_data
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv/eeg_feature_smooth
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/seed_iv/ReadMe.txt
/root/.cache/ka

In [28]:
!pip install tensorflow
import numpy as np
import tensorflow as tf
import scipy
import os
from os import listdir
from scipy.io import loadmat
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras import activations

In [29]:
import os
import numpy as np
from scipy.io import loadmat
directories=["/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/{}".format(i+1) for i in range(3)]
print(directories)

['/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/1', '/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/2', '/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/3']


In [30]:
# EEG channel coordinates
channel_coords = [
    ['0', '0', 'AF3', 'FP1', 'FPZ', 'FP2', 'AF4', '0', '0'],
    ['F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8'],
    ['FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8'],
    ['T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8'],
    ['TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8'],
    ['P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8'],
    ['0', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', '0'],
    ['0', '0', 'CB1', 'O1', 'OZ', 'O2', 'CB2', '0', '0']
]

In [6]:
channel_list = [
    'FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ',
    'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2',
    'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4',
    'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4',
    'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6',
    'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1',
    'O1', 'OZ', 'O2', 'CB2'
]
# Verifying the length of channel coordinates
print(len(channel_coords), len(channel_coords[0]))

8 9


In [11]:
# Initialize the array with the correct shape
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories), len(os.listdir(directories[0])), n, 4, 8, 9, 5, 64))

In [9]:
# Creating a dictionary to map channel names to coordinates
coord_dict = {}
for n in range(len(channel_list)):
    for i, l in enumerate(channel_coords):
        for j, x in enumerate(l):
            if channel_list[n] == x:
                coord_dict[n] = (i, j)
print(coord_dict)

{0: (0, 3), 1: (0, 4), 2: (0, 5), 3: (0, 2), 4: (0, 6), 5: (1, 0), 6: (1, 1), 7: (1, 2), 8: (1, 3), 9: (1, 4), 10: (1, 5), 11: (1, 6), 12: (1, 7), 13: (1, 8), 14: (2, 0), 15: (2, 1), 16: (2, 2), 17: (2, 3), 18: (2, 4), 19: (2, 5), 20: (2, 6), 21: (2, 7), 22: (2, 8), 23: (3, 0), 24: (3, 1), 25: (3, 2), 26: (3, 3), 27: (3, 4), 28: (3, 5), 29: (3, 6), 30: (3, 7), 31: (3, 8), 32: (4, 0), 33: (4, 1), 34: (4, 2), 35: (4, 3), 36: (4, 4), 37: (4, 5), 38: (4, 6), 39: (4, 7), 40: (4, 8), 41: (5, 0), 42: (5, 1), 43: (5, 2), 44: (5, 3), 45: (5, 4), 46: (5, 5), 47: (5, 6), 48: (5, 7), 49: (5, 8), 50: (6, 1), 51: (6, 2), 52: (6, 3), 53: (6, 4), 54: (6, 5), 55: (6, 6), 56: (6, 7), 57: (7, 2), 58: (7, 3), 59: (7, 4), 60: (7, 5), 61: (7, 6)}


In [12]:
# Iterate over directories to load and process data
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(os.path.join(dire, file)) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
      for j in range(n):
            for k, key in enumerate(perSample):
                # Transpose and pad the sample
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0, 2, 1))
                sample = np.pad(sample, [(0, 0), (0, 0), (0, 64 - sample.shape[2])])
                # Fill the array based on channel locations
                for l, channel in enumerate(sample):
                  array[h][i][j][k][coord_dict[l][0]][coord_dict[l][1]] = channel

# Output the shape of the final array
print(array.shape)

/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/1
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/2
/root/.cache/kagglehub/datasets/phhasian0710/seed-iv/versions/1/eeg_feature_smooth/3
(3, 15, 24, 4, 8, 9, 5, 64)


In [13]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])
print(_X.shape)

(1080, 4, 8, 9, 5, 64)


In [14]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}
y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)
print(y.shape)

(1080,)


In [15]:
def fully_conv():
    return tf.keras.Sequential([
    tf.keras.layers.SeparableConv1D(128, 3, data_format='channels_last', padding="same", activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4,activation='softmax')
])

In [16]:
def single_conv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, 3, activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [17]:
def LSTM():
    return tf.keras.Sequential([
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
    ])
def dense():
    return tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [18]:
def twodConv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv2D(
32, 3, padding="same", activation="relu",input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.BatchNormalization(center=True, scale=True),
    tf.keras.layers.Conv2D(
64, 3, padding="same", activation="relu",input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.BatchNormalization(center=True, scale=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [19]:
def threedConv():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv3D(32, kernel_size=5, padding="same", input_shape=X.shape[1:]))
    model.add(tf.keras.layers.BatchNormalization(trainable=False))
    model.add(tf.keras.layers.Activation(activations.relu))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.BatchNormalization(trainable=False))
    model.add(tf.keras.layers.Activation(activations.relu))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10))
    return model

In [20]:
def convlstm2d():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(16, kernel_size=5, padding="same", return_sequences = False, input_shape=X.shape[1:]))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(10))
    return model

In [23]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = fully_conv()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=32)
loss, acc = model.evaluate(X, y)
print(f"Test Accuracy: {acc}")
model.summary()

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)
Epoch 1/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.2676 - loss: 1863.6694
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.3085 - loss: 698.9045
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.3330 - loss: 661.1868
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.3498 - loss: 206.7484
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.3067 - loss: 123.8675
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.3078 - loss: 11.5200
Epoch 7/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.3221 - loss: 35.7911
Epoch 8/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - accuracy: 0.4019 - loss: 5.8660
Epoch 9/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - accuracy: 0.3578 - loss: 5.0809
Epoch 10/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.3517 - loss: 65.0718
Epoch 11/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.2997 - loss: 1371.2950
Epoc

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ separable_conv1d_8 (SeparableConv1D) │ (None, 64, 128)             │         188,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_8 (MaxPooling1D)       │ (None, 32, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_9 (SeparableConv1D) │ (None, 32, 128)             │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_9 (MaxPooling1D)       │ (None, 16, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_10                  │ (None, 16, 128)             │          16,896 │
│ (SeparableConv1D)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_10 (MaxPooling1D)      │ (None, 8, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ separable_conv1d_11                  │ (None, 8, 128)              │          16,896 │
│ (SeparableConv1D)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_11 (MaxPooling1D)      │ (None, 4, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 916,910 (3.50 MB)

 Trainable params: 305,636 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 611,274 (2.33 MB)

In [25]:
model = LSTM()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=32)
loss, acc = model.evaluate(X, y)
print(f"Test Accuracy: {acc}")

Epoch 1/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 187ms/step - accuracy: 0.3193 - loss: 2.0549
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 179ms/step - accuracy: 0.3538 - loss: 1.3377
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 167ms/step - accuracy: 0.3875 - loss: 1.2876
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 12s 216ms/step - accuracy: 0.4267 - loss: 1.2462
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 170ms/step - accuracy: 0.4693 - loss: 1.2127
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 203ms/step - accuracy: 0.4445 - loss: 1.2014
Epoch 7/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 167ms/step - accuracy: 0.4858 - loss: 1.1526
Epoch 8/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 194ms/step - accuracy: 0.4614 - loss: 1.1513
Epoch 9/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 163ms/step - accuracy: 0.4491 - loss: 1.1452
Epoch 10/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 10s 163ms/step - accuracy: 0.4941 - loss: 1.0919
Epoch 11/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 10s 164ms/step - accuracy: 0.4827 - loss: 1.1164
Epoch 12/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 10s 163ms/

In [26]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 100)                 │         616,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 128)                 │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,891,856 (7.22 MB)

 Trainable params: 630,618 (2.41 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,261,238 (4.81 MB)

In [31]:
X = _X.transpose(0, 5, 2,3, 4, 1)[:,:,:,:,:,:]
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], np.prod(X.shape[4:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = convlstm2d()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=32)
loss, acc = model.evaluate(X, y)
print(f"Test Accuracy: {acc}")

(1080, 64, 8, 9, 20)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.3701 - loss: 1.6291
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.4516 - loss: 1.1676
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.4761 - loss: 1.0878
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.4788 - loss: 1.0421
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.5349 - loss: 0.9832
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 86s 2s/step - accuracy: 0.5227 - loss: 0.9815
Epoch 7/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.5401 - loss: 0.9667
Epoch 8/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.5508 - loss: 0.9885
Epoch 9/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.5730 - loss: 0.9297
Epoch 10/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.5594 - loss: 0.9326
Epoch 11/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.5383 - loss: 0.9618
Epoch 12/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.5492 - lo